In [1]:
from lusidtools.jupyter_tools import toggle_code

"""Term Loans and Revolving Credit Facilities

Attributes
----------
TermDeposits
Recipes
Valuations
TransactionTypes
"""

toggle_code("Toggle Docstring")

# Term Loans and Revolving Credit Facilities

This notebook shows you how to manage Revolving Credit Facilities (RCFs) on LUSID using the `TermDeposit` instrument. To illustrate, we walk through the following scenario:

* On the 31 December 2021, the portfolio receives 2 million USD from the fund's "central desk"
* On the 1 Jan 2022, the portfolio makes the 2 million USD available to 3 different clients
* Throughout the next three months, these 3 clients drawdown against their allocation, returning the funds at the end of the quarter
* We offer two types of loans to the clients:
    * Collateralised loans have lower rates
    * Non-collateralised loans have higher rates (to account for the extra risk)
* At the end of the period we calculate the P&L for the fund:
    * Profit is calculated as the interest earned on the amount loaned to each of the three clients
    * The loss is calculated as the interest paid back to "central desk" for borrowing their funds



###  Table of contents

* [1. Notebook setup](#1)
* [2. Recipe setup](#2)
* [3. Create Transaction and Instrument Properties](#3)
* [4. Create fixed rate facility instruments](#4)
* [5. Create portfolio](#5)
* [6. Configure transactions type](#6)
* [7. Book cash from the central desk](#7)
* [8. Allocate cash out to the different clients](#8)
* [9. Drawing funds from the fixed facility](#9)
* [10. Generate cash flow payments](#10)
* [11. Sweep the matured TDs into regular cash](#11)
* [12. Cleanup - delete portfolio](#12)

## Notebook setup <a id='1'></a>

In [3]:
# Import generic non-LUSID packages
import os
import pandas as pd
from datetime import datetime, timedelta
from dateutil.parser import parse
import json
import pytz
from IPython.core.display import HTML

# Import key modules from the LUSID package
import lusid
import lusid.models as lm
import lusid.api as la
from lusid.utilities import ApiClientFactory
from lusidtools.pandas_utils.lusid_pandas import lusid_response_to_data_frame
from lusidtools.cocoon.transaction_type_upload import upsert_transaction_type_alias

# Import key functions from Lusid-Python-Tools and other packages
from lusidjam import RefreshingToken

# Set DataFrame display formats
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.options.display.float_format = "{:,.4f}".format
display(HTML("<style>.container { width:90% !important; }</style>"))

# Set the secrets path
secrets_path = os.getenv("FBN_SECRETS_PATH")

# For running the notebook locally
if secrets_path is None:
    secrets_path = os.path.join(os.path.dirname(os.getcwd()), "secrets.json")

# Authenticate our user and create our API client
api_factory = ApiClientFactory(
    token=RefreshingToken(), api_secrets_filename=secrets_path
)

print("LUSID Environment Initialised")
print(
    "LUSID API Version :",
    api_factory.build(lusid.api.ApplicationMetadataApi)
    .get_lusid_versions()
    .build_version,
)

LUSID Environment Initialised
LUSID API Version : 0.6.10364.0


In [4]:
# Setup the apis we'll use in this notebook:
transaction_portfolios_api = api_factory.build(lusid.api.TransactionPortfoliosApi)
configuration_recipe_api = api_factory.build(lusid.ConfigurationRecipeApi)
instruments_api = api_factory.build(lusid.InstrumentsApi)
property_definition_api = api_factory.build(lusid.PropertyDefinitionsApi)
portfolios_api = api_factory.build(lusid.api.PortfoliosApi)
transaction_configuration_api = api_factory.build(lusid.api.TransactionConfigurationApi)

In [5]:
# Set the scope we'll use in this notebook:
scope = "rcf-lifecycle"

In [6]:
# Helper tools
def get_holdings_df(trading_code, trading_scope, effective_at):

    holdings = transaction_portfolios_api.get_holdings(
        scope=trading_scope,
        code=trading_code,
        effective_at=effective_at,
        property_keys=["Instrument/default/Name", "Instrument/default/ClientInternal"],
    )

    return lusid_response_to_data_frame(holdings, rename_properties=True)

## Recipe setup <a id='2'></a>

Here we create a recipe which will be used to calculate the coupon payments on the `TermDeposit`. We use the "constant time value of money" model to calculate these coupons. In other words, no discounting is applied.

In [7]:
recipe_code = "rcf-recipe"
recipe_scope = scope
price_field = "mid"

# Create a recipe to perform a valuation
configuration_recipe = lm.ConfigurationRecipe(
    scope=recipe_scope,
    code=recipe_code,
    market=lm.MarketContext(
        market_rules=[
            # define how to resolve the quotes
            lm.MarketDataKeyRule(
                key="Equity.ClientInternal.*",
                supplier="Lusid",
                data_scope=recipe_scope,
                quote_type="Price",
                quote_interval="2D.0D",
                field="mid",
            ),
            lm.MarketDataKeyRule(
                key="Equity.RIC.*",
                supplier="Lusid",
                data_scope=recipe_scope,
                quote_type="Rate",
                quote_interval="2D.0D",
                field=price_field,
            ),
            lm.MarketDataKeyRule(
                key="Equity.RIC.*",
                supplier="Lusid",
                data_scope=recipe_scope,
                quote_interval="2D.0D",
                quote_type="Price",
                field=price_field,
            ),
        ],
        options=lm.MarketOptions(
            default_supplier="Lusid",
            default_instrument_code_type="Isin",
            default_scope=recipe_scope,
            calendar_scope="CoppClarkHolidayCalendars",
            convention_scope="Conventions",
        ),
    ),
    pricing=lm.PricingContext(
        options={"AllowPartiallySuccessfulEvaluation": False},
        model_rules=[
            lm.VendorModelRule(
                supplier="Lusid",
                model_name="ConstantTimeValueOfMoney",
                instrument_type="TermDeposit",
                parameters="{}",
            ),
            lm.VendorModelRule(
                supplier="Lusid",
                model_name="ConstantTimeValueOfMoney",
                instrument_type="FundingLeg",
                parameters="{}",
            ),
        ],
    ),
    holding=lm.HoldingContext(tax_lot_level_holdings=True),
)

upsert_configuration_recipe_response = (
    configuration_recipe_api.upsert_configuration_recipe(
        upsert_recipe_request=lm.UpsertRecipeRequest(
            configuration_recipe=configuration_recipe
        )
    )
)

## Create Transaction and Instrument Properties  <a id='3'></a>

In this section we create some properties. These are the user defined extensions to LUSID's standard data model. 

In [8]:
def create_property(domain, property_code, property_name, data_type):

    # Defines the property definition
    request = lusid.models.PropertyDefinition(
        domain=domain,
        scope=scope,
        code=property_code,
        display_name=property_name,
        data_type_id=lusid.models.ResourceId(scope="system", code=data_type),
    )

    # Creates and upserts the property definition
    try:
        api_factory.build(lusid.PropertyDefinitionsApi).create_property_definition(
            request
        )
        print("Property definition created.")
    except lusid.ApiException as e:
        if json.loads(e.body)["name"] == "PropertyAlreadyExists":
            print(f"Property Transaction/{scope}/{property_code} already exists")
        else:
            print(e.body)

In [9]:
property_details = [
    ("Instrument", "facility_type", "Facility Type", "string"),
    ("Instrument", "collateral_type", "collateral_type", "string"),
    ("Instrument", "loan_origin", "loan_origin", "string"),
    ("Transaction", "client_id", "Holder Identifier", "string"),
    ("Transaction", "draw_from", "draw_from", "string"),
    ("Transaction", "balance_type", "balance_type", "string"),
    ("Transaction", "draw_status", "draw_status", "string"),
]

for domain, property_code, property_name, data_type in property_details:

    create_property(domain, property_code, property_name, data_type)

Property Transaction/rcf-lifecycle/facility_type already exists
Property Transaction/rcf-lifecycle/collateral_type already exists
Property Transaction/rcf-lifecycle/loan_origin already exists
Property Transaction/rcf-lifecycle/client_id already exists
Property Transaction/rcf-lifecycle/draw_from already exists
Property Transaction/rcf-lifecycle/balance_type already exists
Property Transaction/rcf-lifecycle/draw_status already exists


## Create fixed rate facility instruments  <a id='4'></a>

In this section we created a fixed rate facility using the TermDeposit model in LUSID. Specifically, we create a set of USD Term Deposits. The origin column distinguishes between:

* <b>Central desk</b> - the funds received into the portfolio from the central desk
* <b>Portfolio </b> - the funds which our portfolio will lend out to clients

In [10]:
rev_df = pd.read_excel("data/rcf-data.xlsx", sheet_name="fixed_facility")
rev_df

,instrument_name,identifier,start_date,maturity_date,rate_or_spread,payment_frequency,currency,facility_type,collateral_type,loan_origin
0,REV-01-Jan-Deposit-3M,REV0000-FIX,2022-01-01T00:00:00Z,2022-04-01T00:00:00Z,0.0200,1M,USD,Undrawn,Collateralised,CentralDesk
1,REV-01-Feb-Drawn-2M,REV0002-FIX,2022-02-01T00:00:00Z,2022-04-01T00:00:00Z,0.0650,1M,USD,Drawn,UnCollateralised,Portfolio
2,REV-01-Feb-Drawn-2M,REV0002-CFIX,2022-02-01T00:00:00Z,2022-04-01T00:00:00Z,0.0430,1M,USD,Drawn,Collateralised,Portfolio
3,REV-14-Jan-Drawn-2M,REV0003-FIX,2022-01-14T00:00:00Z,2022-03-14T00:00:00Z,0.0750,1M,USD,Drawn,UnCollateralised,Portfolio
4,REV-14-Jan-Drawn-2M,REV0003-CFIX,2022-01-14T00:00:00Z,2022-03-14T00:00:00Z,0.0530,1M,USD,Drawn,Collateralised,Portfolio


In [11]:
for index, row in rev_df.iterrows():

    term_deposit_instrument = lusid.models.TermDeposit(
        start_date=row.start_date,
        maturity_date=row.maturity_date,
        contract_size=1,
        flow_convention=lusid.models.FlowConventions(
            currency=row.currency,
            payment_frequency=row.payment_frequency,
            roll_convention="MF",
            day_count_convention="Act360",
            payment_calendars=["USD"],
            reset_calendars=["USD"],
            settle_days=0,
            reset_days=0,
        ),
        rate=row.rate_or_spread,
        instrument_type="TermDeposit",
    )

    properties = [
        lm.ModelProperty(
            key=f"Instrument/{scope}/facility_type",
            value=lm.PropertyValue(label_value=row.facility_type),
        ),
        lm.ModelProperty(
            key=f"Instrument/{scope}/collateral_type",
            value=lm.PropertyValue(label_value=row.collateral_type),
        ),
    ]

    # Creates the instrument definition
    instrument_definition = lusid.models.InstrumentDefinition(
        name=row.instrument_name,
        properties=properties,
        identifiers={"ClientInternal": lusid.models.InstrumentIdValue(row.identifier)},
        definition=term_deposit_instrument,
    )

    upsert_request = {row.identifier: instrument_definition}

    # Upserts the instrument to LUSID
    response = api_factory.build(lusid.api.InstrumentsApi).upsert_instruments(
        request_body=upsert_request
    )

## Create portfolio  <a id='5'></a>

Next we create a portfolio to hold the different loans.

In [12]:
# Setup scope and code for the portfolio
trading_scope = scope
trading_code = "Revolving-Credit-Facility"

In [13]:
try:

    transaction_portfolios_api.create_portfolio(
        scope=trading_scope,
        create_transaction_portfolio_request=lm.CreateTransactionPortfolioRequest(
            display_name=trading_code,
            code=trading_code,
            base_currency="USD",
            created="2010-01-01",
            sub_holding_keys=[
                f"Transaction/{scope}/client_id",
                f"Transaction/{scope}/balance_type",
                f"Transaction/{scope}/draw_status",
            ],
        ),
    )
except lusid.ApiException as e:

    print(json.loads(e.body)["title"])

## Configure transactions types  <a id='6'></a>

Next we configure a set of Transaction Types to manage the RCF lifecycle:

<!-- <b>TransactionType: DrawFunds</b> -->

#### DrawFunds

* This is used to draw funds from the "Central Desk" allocation line
* This transaction type has three movements:
    1. Move cash out of the "Undrawn" credit facility line 
    2. Move cash to the "Drawn" credit facility line
    3. Create a new line to represent the amount lent out to the client
* Each of these movements dynamically updates the "Draw Status" and "Balance Type" of each line
* This Transaction Type also requires a custom side `RcfDrawdown` which captures the drawdown security from the transaction



In [14]:
drawndown_id = f"Transaction/{scope}/draw_from"

# Define the custom side
side_definition = lm.SideDefinitionRequest(
    security=drawndown_id,
    currency="Txn:SettlementCurrency",
    rate="Txn:TradeToPortfolioRate",
    units="Txn:Units",
    amount="Txn:TradeAmount",
)

response = transaction_configuration_api.set_side_definition(
    side="RcfDrawdown",
    side_definition_request=side_definition,
)

In [15]:
new_transaction_config = []

new_transaction_config.append(
    lm.TransactionConfigurationDataRequest(
        aliases=[
            lm.TransactionConfigurationTypeAlias(
                type="DrawFunds",
                description="Adding new undrawn funds to the portfolio",
                transaction_class="default",
                transaction_group="default",
                transaction_roles="Longer",
            )
        ],
        movements=[
            lm.TransactionConfigurationMovementDataRequest(
                movement_types="StockMovement",
                side="Side1",
                direction=1,
                properties={},
                mappings=[
                    lm.TransactionPropertyMappingRequest(
                        property_key=f"Transaction/{scope}/draw_status", set_to="Drawn"
                    ),
                    lm.TransactionPropertyMappingRequest(
                        property_key=f"Transaction/{scope}/balance_type",
                        set_to="CreditFacility",
                    ),
                ],
            ),
            lm.TransactionConfigurationMovementDataRequest(
                movement_types="StockMovement",
                side="Side1",
                direction=-1,
                properties={},
                mappings=[
                    lm.TransactionPropertyMappingRequest(
                        property_key=f"Transaction/{scope}/draw_status",
                        set_to="Undrawn",
                    ),
                    lm.TransactionPropertyMappingRequest(
                        property_key=f"Transaction/{scope}/balance_type",
                        set_to="CreditFacility",
                    ),
                ],
            ),
            lm.TransactionConfigurationMovementDataRequest(
                movement_types="StockMovement",
                side="RcfDrawdown",
                direction=-1,
                properties={},
                mappings=[
                    lm.TransactionPropertyMappingRequest(
                        property_key=f"Transaction/{scope}/balance_type",
                        set_to="BorrowedAmount",
                    ),
                    lm.TransactionPropertyMappingRequest(
                        property_key=f"Transaction/{scope}/draw_status", set_to="Drawn"
                    ),
                ],
            ),
        ],
        properties={},
    ),
)

#### UndrawnAllocation

* This is used to allocate undrawn funds from the "central desk" to the different clients
* This transaction type has two movements:
    1. Take cash out of the inital funds bucket
    2. Allocate an "undrawn" loan amount to each individual client


In [16]:
new_transaction_config.append(
    lm.TransactionConfigurationDataRequest(
        aliases=[
            lm.TransactionConfigurationTypeAlias(
                type="UndrawnAllocation",
                description="A pro-rated repayment transaction",
                transaction_class="default",
                transaction_group="default",
                transaction_roles="Longer",
            )
        ],
        movements=[
            lm.TransactionConfigurationMovementDataRequest(
                movement_types="StockMovement",
                side="Side1",
                direction=1,
                properties={},
                mappings=[],
            ),
            lm.TransactionConfigurationMovementDataRequest(
                movement_types="CashCommitment",
                side="Side2",
                direction=-1,
                properties={},
                mappings=[
                    lm.TransactionPropertyMappingRequest(
                        property_key=f"Transaction/{scope}/client_id",
                        set_to="NewFunds",
                    ),
                    lm.TransactionPropertyMappingRequest(
                        property_key=f"Transaction/{scope}/draw_status",
                        set_to="NewFunds",
                    ),
                    lm.TransactionPropertyMappingRequest(
                        property_key=f"Transaction/{scope}/balance_type",
                        set_to="NewFunds",
                    ),
                ],
            ),
        ],
        properties={},
    ),
)

#### MatDeposit

* This is used to mature the P&L from the TDs at end of period
* This transaction type has two movements:
    1. Flatten the TD position
    2. Convert the TD position to regular cash


In [17]:
new_transaction_config.append(
    lm.TransactionConfigurationDataRequest(
        aliases=[
            lm.TransactionConfigurationTypeAlias(
                type="MatDeposit",
                description="A pro-rated repayment transaction",
                transaction_class="default",
                transaction_group="default",
                transaction_roles="Longer",
            )
        ],
        movements=[
            lm.TransactionConfigurationMovementDataRequest(
                movement_types="StockMovement",
                side="Side1",
                direction=1,
                properties={},
                mappings=[],
            ),
            lm.TransactionConfigurationMovementDataRequest(
                movement_types="CashCommitment",
                side="Side2",
                direction=1,
                properties={},
                mappings=[
                    lm.TransactionPropertyMappingRequest(
                        property_key=f"Transaction/{scope}/client_id", set_to="Cash"
                    ),
                    lm.TransactionPropertyMappingRequest(
                        property_key=f"Transaction/{scope}/draw_status", set_to="Cash"
                    ),
                    lm.TransactionPropertyMappingRequest(
                        property_key=f"Transaction/{scope}/balance_type", set_to="Cash"
                    ),
                ],
            ),
        ],
        properties={},
    ),
)

In [18]:
# Upload the transaction type
new_txn_config = upsert_transaction_type_alias(
    api_factory, new_transaction_config=new_transaction_config
)

## Book cash from the central desk  <a id='7'></a>

In this first step, we receive 2 million USD from the bank's "Cental desk":

* This cash is booked into the fund as regular CCY_USD cash
* We set balance_type, draw_status and client_id to "NewFunds"

In [19]:
initial_funds = pd.read_excel("data/rcf-data.xlsx", sheet_name="initial_funds")
initial_funds

,type,trade_date,quantity,identifier,transaction_id,client_id,balance_type,draw_status,cash_transaction,draw_from
0,FundsIn,2021-12-31T00:00:00Z,2000000,CCY_USD,new_funds_001,NewFunds,NewFunds,NewFunds,Y,NoValue


In [20]:
for index, row in initial_funds.iterrows():

    upsert_transactions = transaction_portfolios_api.upsert_transactions(
                scope=trading_scope,
                code=trading_code,
                transaction_request=[
                    lm.TransactionRequest(
                        transaction_id=row.transaction_id,
                        type=row.type,
                        instrument_identifiers={"Instrument/default/LusidInstrumentId": row.identifier},
                        transaction_date=row.trade_date,
                        settlement_date=row.trade_date,
                        units=row.quantity,
                        transaction_price=lm.TransactionPrice(price=1),
                        total_consideration=lm.CurrencyAndAmount(
                            amount=row.quantity, currency="USD"
                        ),
                        exchange_rate=1,
                        transaction_currency="USD",
                        properties={
                            f"Transaction/{scope}/balance_type": lm.ModelProperty(
                                key=f"Transaction/{scope}/balance_type",
                                value=lm.PropertyValue(label_value="NewFunds"),
                            ),
                            f"Transaction/{scope}/client_id": lm.ModelProperty(
                                key=f"Transaction/{scope}/client_id",
                                value=lm.PropertyValue(label_value="NewFunds"),
                            ),
                            f"Transaction/{scope}/draw_status": lm.ModelProperty(
                                key=f"Transaction/{scope}/draw_status",
                                value=lm.PropertyValue(label_value="NewFunds"),
                            )
                        },
                    )
                ],
            )
    
    print(f"Upserted transaction {row.transaction_id} on {row.identifier}")

Upserted transaction new_funds_001 on CCY_USD


In [21]:
get_holdings_df(trading_code, trading_scope, "2021-12-31T00:00:00Z")

,instrument_scope,instrument_uid,draw_status(rcf-lifecycle-SubHoldingKeys),client_id(rcf-lifecycle-SubHoldingKeys),balance_type(rcf-lifecycle-SubHoldingKeys),Name(default-Properties),SourcePortfolioId(default-Properties),SourcePortfolioScope(default-Properties),holding_type,units,settled_units,cost.amount,cost.currency,cost_portfolio_ccy.amount,cost_portfolio_ccy.currency,currency,holding_type_name
0,default,CCY_USD,NewFunds,NewFunds,NewFunds,USD,Revolving-Credit-Facility,rcf-lifecycle,B,"2,000,000.0000","2,000,000.0000","2,000,000.0000",USD,"2,000,000.0000",USD,USD,Balance


## Allocate cash out to the different clients  <a id='8'></a>

Next we allocate the 2 million USD across 3 different clients. At this point the clients have not yet taken a loan from the portfolio. Rather we have made some funds available which they can drawdown as they require.

In [22]:
undrawn_allocations = pd.read_excel(
    "data/rcf-data.xlsx", sheet_name="undrawn_allocation"
)
undrawn_allocations["cash_transaction"] = "N"
undrawn_allocations

,type,currency,trade_date,client_id,percentage,weight,quantity,identifier,transaction_id,balance_type,draw_status,cash_transaction
0,UndrawnAllocation,USD,2022-01-01T00:00:00Z,Client-1,0.2500,0.2500,500000,REV0000-FIX,alloc_001,CreditFacility,Undrawn,N
1,UndrawnAllocation,USD,2022-01-01T00:00:00Z,Client-2,0.2500,0.2500,500000,REV0000-FIX,alloc_002,CreditFacility,Undrawn,N
2,UndrawnAllocation,USD,2022-01-01T00:00:00Z,Client-3,0.5000,0.5000,1000000,REV0000-FIX,alloc_003,CreditFacility,Undrawn,N


In [23]:
for index, row in undrawn_allocations.iterrows():

    upsert_transactions = transaction_portfolios_api.upsert_transactions(
                scope=trading_scope,
                code=trading_code,
                transaction_request=[
                    lm.TransactionRequest(
                        transaction_id=row.transaction_id,
                        type=row.type,
                        instrument_identifiers={"Instrument/default/ClientInternal": row.identifier},
                        transaction_date=row.trade_date,
                        settlement_date=row.trade_date,
                        units=row.quantity,
                        transaction_price=lm.TransactionPrice(price=1),
                        total_consideration=lm.CurrencyAndAmount(
                            amount=row.quantity, currency="USD"
                        ),
                        exchange_rate=1,
                        transaction_currency="USD",
                        properties={
                            f"Transaction/{scope}/balance_type": lm.ModelProperty(
                                key=f"Transaction/{scope}/balance_type",
                                value=lm.PropertyValue(label_value=row.balance_type),
                            ),
                            f"Transaction/{scope}/client_id": lm.ModelProperty(
                                key=f"Transaction/{scope}/client_id",
                                value=lm.PropertyValue(label_value=row.client_id),
                            ),
                            f"Transaction/{scope}/draw_status": lm.ModelProperty(
                                key=f"Transaction/{scope}/draw_status",
                                value=lm.PropertyValue(label_value=row.draw_status),
                            )
                        },
                    )
                ],
            )
    
    print(f"Upserted transaction {row.transaction_id} on {row.identifier}")

Upserted transaction alloc_001 on REV0000-FIX
Upserted transaction alloc_002 on REV0000-FIX
Upserted transaction alloc_003 on REV0000-FIX


In [24]:
get_holdings_df(trading_code, trading_scope, "2022-01-01T00:00:00Z")

,instrument_scope,instrument_uid,client_id(rcf-lifecycle-SubHoldingKeys),draw_status(rcf-lifecycle-SubHoldingKeys),balance_type(rcf-lifecycle-SubHoldingKeys),Name(default-Properties),ClientInternal(default-Properties),SourcePortfolioId(default-Properties),SourcePortfolioScope(default-Properties),holding_type,units,settled_units,cost.amount,cost.currency,cost_portfolio_ccy.amount,cost_portfolio_ccy.currency,currency,holding_type_name
0,default,LUID_00003FKG,Client-1,Undrawn,CreditFacility,REV-01-Jan-Deposit-3M,REV0000-FIX,Revolving-Credit-Facility,rcf-lifecycle,P,"500,000.0000","500,000.0000","500,000.0000",USD,"500,000.0000",USD,USD,Position
1,default,LUID_00003FKG,Client-2,Undrawn,CreditFacility,REV-01-Jan-Deposit-3M,REV0000-FIX,Revolving-Credit-Facility,rcf-lifecycle,P,"500,000.0000","500,000.0000","500,000.0000",USD,"500,000.0000",USD,USD,Position
2,default,LUID_00003FKG,Client-3,Undrawn,CreditFacility,REV-01-Jan-Deposit-3M,REV0000-FIX,Revolving-Credit-Facility,rcf-lifecycle,P,"1,000,000.0000","1,000,000.0000","1,000,000.0000",USD,"1,000,000.0000",USD,USD,Position


## Drawing funds from the fixed facility  <a id='9'></a>

Throughout the quarter, the three clients drawdown against their revolving credit facility. In LUSID, we process these as `DrawFund` transactions.

In [25]:
draw_funds = pd.read_excel("data/rcf-data.xlsx", sheet_name="drawdown")

draw_funds["draw_from"] = draw_funds["draw_from"].apply(
    lambda x: instruments_api.get_instrument("ClientInternal", x).lusid_instrument_id
)

draw_funds

,type,trade_date,client_id,quantity,identifier,draw_from,transaction_id
0,DrawFunds,2022-02-01T00:00:00Z,Client-1,300000,REV0000-FIX,LUID_00003FKI,draw_001
1,DrawFunds,2022-02-01T00:00:00Z,Client-2,300000,REV0000-FIX,LUID_00003FKI,draw_002
2,DrawFunds,2022-02-01T00:00:00Z,Client-3,600000,REV0000-FIX,LUID_00003FKJ,draw_003
3,DrawFunds,2022-01-14T00:00:00Z,Client-1,100000,REV0000-FIX,LUID_00003FKK,draw_004
4,DrawFunds,2022-01-14T00:00:00Z,Client-2,100000,REV0000-FIX,LUID_00003FKK,draw_005
5,DrawFunds,2022-01-14T00:00:00Z,Client-3,120000,REV0000-FIX,LUID_00003FKL,draw_006


In [26]:
for index, row in draw_funds.iterrows():

    upsert_transactions = transaction_portfolios_api.upsert_transactions(
                scope=trading_scope,
                code=trading_code,
                transaction_request=[
                    lm.TransactionRequest(
                        transaction_id=row.transaction_id,
                        type=row.type,
                        instrument_identifiers={"Instrument/default/ClientInternal": row.identifier},
                        transaction_date=row.trade_date,
                        settlement_date=row.trade_date,
                        units=row.quantity,
                        transaction_price=lm.TransactionPrice(price=1),
                        total_consideration=lm.CurrencyAndAmount(
                            amount=row.quantity, currency="USD"
                        ),
                        exchange_rate=1,
                        transaction_currency="USD",
                        properties={
                            f"Transaction/{scope}/client_id": lm.ModelProperty(
                                key=f"Transaction/{scope}/client_id",
                                value=lm.PropertyValue(label_value=row.client_id),
                            ),           
                            f"Transaction/{scope}/draw_from": lm.ModelProperty(
                                key=f"Transaction/{scope}/draw_from",
                                value=lm.PropertyValue(label_value=row.draw_from),
                            )
                        },
                    )
                ],
            )
    
    print(f"Upserted transaction {row.transaction_id} on {row.identifier}")

Upserted transaction draw_001 on REV0000-FIX
Upserted transaction draw_002 on REV0000-FIX
Upserted transaction draw_003 on REV0000-FIX
Upserted transaction draw_004 on REV0000-FIX
Upserted transaction draw_005 on REV0000-FIX
Upserted transaction draw_006 on REV0000-FIX


In [27]:
get_holdings_df(trading_code, trading_scope, "2022-02-01")

,instrument_scope,instrument_uid,client_id(rcf-lifecycle-SubHoldingKeys),draw_status(rcf-lifecycle-SubHoldingKeys),balance_type(rcf-lifecycle-SubHoldingKeys),Name(default-Properties),ClientInternal(default-Properties),SourcePortfolioId(default-Properties),SourcePortfolioScope(default-Properties),holding_type,units,settled_units,cost.amount,cost.currency,cost_portfolio_ccy.amount,cost_portfolio_ccy.currency,currency,holding_type_name
0,default,LUID_00003FKG,Client-1,Undrawn,CreditFacility,REV-01-Jan-Deposit-3M,REV0000-FIX,Revolving-Credit-Facility,rcf-lifecycle,P,"100,000.0000","100,000.0000","100,000.0000",USD,"100,000.0000",USD,USD,Position
1,default,LUID_00003FKG,Client-2,Undrawn,CreditFacility,REV-01-Jan-Deposit-3M,REV0000-FIX,Revolving-Credit-Facility,rcf-lifecycle,P,"100,000.0000","100,000.0000","100,000.0000",USD,"100,000.0000",USD,USD,Position
2,default,LUID_00003FKG,Client-3,Undrawn,CreditFacility,REV-01-Jan-Deposit-3M,REV0000-FIX,Revolving-Credit-Facility,rcf-lifecycle,P,"280,000.0000","280,000.0000","280,000.0000",USD,"280,000.0000",USD,USD,Position
3,default,LUID_00003FKG,Client-1,Drawn,CreditFacility,REV-01-Jan-Deposit-3M,REV0000-FIX,Revolving-Credit-Facility,rcf-lifecycle,P,"400,000.0000","400,000.0000","400,000.0000",USD,"400,000.0000",USD,USD,Position
4,default,LUID_00003FKG,Client-2,Drawn,CreditFacility,REV-01-Jan-Deposit-3M,REV0000-FIX,Revolving-Credit-Facility,rcf-lifecycle,P,"400,000.0000","400,000.0000","400,000.0000",USD,"400,000.0000",USD,USD,Position
5,default,LUID_00003FKG,Client-3,Drawn,CreditFacility,REV-01-Jan-Deposit-3M,REV0000-FIX,Revolving-Credit-Facility,rcf-lifecycle,P,"720,000.0000","720,000.0000","720,000.0000",USD,"720,000.0000",USD,USD,Position
6,default,LUID_00003FKK,Client-1,Drawn,BorrowedAmount,REV-14-Jan-Drawn-2M,REV0003-FIX,Revolving-Credit-Facility,rcf-lifecycle,P,"-100,000.0000","-100,000.0000","-100,000.0000",USD,"-100,000.0000",USD,USD,Position
7,default,LUID_00003FKK,Client-2,Drawn,BorrowedAmount,REV-14-Jan-Drawn-2M,REV0003-FIX,Revolving-Credit-Facility,rcf-lifecycle,P,"-100,000.0000","-100,000.0000","-100,000.0000",USD,"-100,000.0000",USD,USD,Position
8,default,LUID_00003FKL,Client-3,Drawn,BorrowedAmount,REV-14-Jan-Drawn-2M,REV0003-CFIX,Revolving-Credit-Facility,rcf-lifecycle,P,"-120,000.0000","-120,000.0000","-120,000.0000",USD,"-120,000.0000",USD,USD,Position
9,default,LUID_00003FKI,Client-1,Drawn,BorrowedAmount,REV-01-Feb-Drawn-2M,REV0002-FIX,Revolving-Credit-Facility,rcf-lifecycle,P,"-300,000.0000","-300,000.0000","-300,000.0000",USD,"-300,000.0000",USD,USD,Position


## Generate cash flow payments  <a id='10'></a>

At the end of the quarter, we process the payments. From a high-level there are two sets of payments:
    
1. The portfolio pays back the 2 million USD to the "Central Desk" (with some interest)
2. The portfolio collects loans back from the clients (with interest)

The P&L is then calculated as the difference between interest paid and interest received.

In [28]:
start_date = "2021-01-01T00:00:00Z"
end_date = "2022-12-31T00:00:00Z"

In [29]:
# Upsertable cash flows call for interest payments
cfs = transaction_portfolios_api.get_upsertable_portfolio_cash_flows(
    scope=trading_scope,
    code=trading_code,
    effective_at=end_date,
    window_start=start_date,
    window_end=end_date,
    recipe_id_scope=recipe_scope, 
    recipe_id_code=recipe_code, 
)

df_cf = lusid_response_to_data_frame(cfs)
display(df_cf)

,transaction_id,type,instrument_identifiers.Instrument/default/LusidInstrumentId,instrument_scope,instrument_uid,transaction_date,settlement_date,units,transaction_price.price,transaction_price.type,total_consideration.amount,total_consideration.currency,exchange_rate,transaction_currency,properties.Transaction/rcf-lifecycle/balance_type.key,properties.Transaction/rcf-lifecycle/balance_type.value.label_value,properties.Transaction/rcf-lifecycle/draw_status.key,properties.Transaction/rcf-lifecycle/draw_status.value.label_value,properties.Transaction/rcf-lifecycle/client_id.key,properties.Transaction/rcf-lifecycle/client_id.value.label_value,properties.Transaction/default/ParentLuid.key,properties.Transaction/default/ParentLuid.value.label_value,source,entry_date_time,transaction_status
0,draw_001-LUID_00003FKG-20220401-Cash-USD-Receive,CashFlow,LUID_00003FKG,default,LUID_00003FKG,2022-04-01 00:00:00+00:00,2022-04-01 00:00:00+00:00,"100,500.0000",1.0000,Price,"100,500.0000",USD,1.0000,USD,Transaction/rcf-lifecycle/balance_type,CreditFacility,Transaction/rcf-lifecycle/draw_status,Undrawn,Transaction/rcf-lifecycle/client_id,Client-1,Transaction/default/ParentLuid,LUID_00003FKG,default,0001-01-01 00:00:00+00:00,Active
1,draw_002-LUID_00003FKG-20220401-Cash-USD-Receive,CashFlow,LUID_00003FKG,default,LUID_00003FKG,2022-04-01 00:00:00+00:00,2022-04-01 00:00:00+00:00,"100,500.0000",1.0000,Price,"100,500.0000",USD,1.0000,USD,Transaction/rcf-lifecycle/balance_type,CreditFacility,Transaction/rcf-lifecycle/draw_status,Undrawn,Transaction/rcf-lifecycle/client_id,Client-2,Transaction/default/ParentLuid,LUID_00003FKG,default,0001-01-01 00:00:00+00:00,Active
2,draw_003-LUID_00003FKG-20220401-Cash-USD-Receive,CashFlow,LUID_00003FKG,default,LUID_00003FKG,2022-04-01 00:00:00+00:00,2022-04-01 00:00:00+00:00,"281,400.0000",1.0000,Price,"281,400.0000",USD,1.0000,USD,Transaction/rcf-lifecycle/balance_type,CreditFacility,Transaction/rcf-lifecycle/draw_status,Undrawn,Transaction/rcf-lifecycle/client_id,Client-3,Transaction/default/ParentLuid,LUID_00003FKG,default,0001-01-01 00:00:00+00:00,Active
3,draw_001-LUID_00003FKG-20220401-Cash-USD-Receive,CashFlow,LUID_00003FKG,default,LUID_00003FKG,2022-04-01 00:00:00+00:00,2022-04-01 00:00:00+00:00,"402,000.0000",1.0000,Price,"402,000.0000",USD,1.0000,USD,Transaction/rcf-lifecycle/balance_type,CreditFacility,Transaction/rcf-lifecycle/draw_status,Drawn,Transaction/rcf-lifecycle/client_id,Client-1,Transaction/default/ParentLuid,LUID_00003FKG,default,0001-01-01 00:00:00+00:00,Active
4,draw_002-LUID_00003FKG-20220401-Cash-USD-Receive,CashFlow,LUID_00003FKG,default,LUID_00003FKG,2022-04-01 00:00:00+00:00,2022-04-01 00:00:00+00:00,"402,000.0000",1.0000,Price,"402,000.0000",USD,1.0000,USD,Transaction/rcf-lifecycle/balance_type,CreditFacility,Transaction/rcf-lifecycle/draw_status,Drawn,Transaction/rcf-lifecycle/client_id,Client-2,Transaction/default/ParentLuid,LUID_00003FKG,default,0001-01-01 00:00:00+00:00,Active
5,draw_003-LUID_00003FKG-20220401-Cash-USD-Receive,CashFlow,LUID_00003FKG,default,LUID_00003FKG,2022-04-01 00:00:00+00:00,2022-04-01 00:00:00+00:00,"723,600.0000",1.0000,Price,"723,600.0000",USD,1.0000,USD,Transaction/rcf-lifecycle/balance_type,CreditFacility,Transaction/rcf-lifecycle/draw_status,Drawn,Transaction/rcf-lifecycle/client_id,Client-3,Transaction/default/ParentLuid,LUID_00003FKG,default,0001-01-01 00:00:00+00:00,Active
6,draw_004-LUID_00003FKK-20220314-Cash-USD-Receive,CashFlow,LUID_00003FKK,default,LUID_00003FKK,2022-03-14 00:00:00+00:00,2022-03-14 00:00:00+00:00,"-101,229.1667",1.0000,Price,"-101,229.1667",USD,1.0000,USD,Transaction/rcf-lifecycle/balance_type,BorrowedAmount,Transaction/rcf-lifecycle/draw_status,Drawn,Transaction/rcf-lifecycle/client_id,Client-1,Transaction/default/ParentLuid,LUID_00003FKK,default,0001-01-01 00:00:00+00:00,Active
7,draw_005-LUID_00003FKK-20220314-Cash-USD-Receive,CashFlow,LUID_00003FKK,default,LUID_00003FKK,2022-03-14 00:00:00+00:00,2022-03-14 00:00:00+00:00,"-1

In [30]:
def upsert_cfs_from_tds(cfs_values):

    for count, value in enumerate(cfs_values):

        value.units = value.units * -1

        value.transaction_id = f"cf_trd_00{count}"

    # Add cash interest transactions
    upsert_transactions_response = transaction_portfolios_api.upsert_transactions(
        scope=trading_scope, code=trading_code, transaction_request=cfs.values
    )

    return upsert_transactions_response

In [31]:
upsert_cfs = upsert_cfs_from_tds(cfs.values)

## Sweep the matured TDs into regular cash  <a id='11'></a>

Finally, we sweep the TD positions into regular USD cash:

In [32]:
matured_td_holdings = transaction_portfolios_api.get_holdings(
    scope=trading_scope, code=trading_code, effective_at="2022-04-04T00:00:00Z"
)


In [33]:
mat_date = "2022-04-02T09:00:00Z"

for count, holding in enumerate(matured_td_holdings.values):

    properties = holding.sub_holding_keys
    trd_id = "mat_trd_" + str(count)
    luid = holding.instrument_uid
    currency = holding.currency
    units = holding.units
    cost = holding.cost.amount

    transaction_portfolios_api.upsert_transactions(
        scope=trading_scope,
        code=trading_code,
        transaction_request=[
            lm.TransactionRequest(
                transaction_id=trd_id,
                type="MatDeposit",
                instrument_identifiers={"Instrument/default/LusidInstrumentId": luid},
                transaction_date=mat_date,
                settlement_date=mat_date,
                units=units * -1,
                transaction_price=lm.TransactionPrice(price=1),
                total_consideration=lm.CurrencyAndAmount(
                    amount=units, currency=currency
                ),
                exchange_rate=1,
                transaction_currency=currency,
                properties=properties,
            )
        ],
    )
    
    print(f"Upserted transaction {trd_id} on {luid}")

Upserted transaction mat_trd_0 on LUID_00003FKG
Upserted transaction mat_trd_1 on LUID_00003FKG
Upserted transaction mat_trd_2 on LUID_00003FKG
Upserted transaction mat_trd_3 on LUID_00003FKG
Upserted transaction mat_trd_4 on LUID_00003FKG
Upserted transaction mat_trd_5 on LUID_00003FKG
Upserted transaction mat_trd_6 on LUID_00003FKK
Upserted transaction mat_trd_7 on LUID_00003FKK
Upserted transaction mat_trd_8 on LUID_00003FKL
Upserted transaction mat_trd_9 on LUID_00003FKI
Upserted transaction mat_trd_10 on LUID_00003FKI
Upserted transaction mat_trd_11 on LUID_00003FKJ


## Cleanup - delete portfolio   <a id='12'></a>

In [34]:
# delete_portfolio = portfolios_api.delete_portfolio(scope=trading_scope, code=trading_code)